# Looking at the amzn data that did not join

Making sure there wasn't any data lost because the titles weren't good enough an "id" to join on

In [1]:
import duckdb as db
import pandas as pd

In [2]:
conn = db.connect('../data/books.duckdb')
cursor = conn.cursor()

In [3]:
q = cursor.execute('PRAGMA database_size;')
res = q.fetchall()
cols = ['database_size', 'block_size', 'total_blocks', 'used_blocks'
        , 'free_blocks', 'wal_size', 'memory_usage', 'memory_limit']
for k, v in zip(cols, res[0]):
    print(k, ':',  v)

database_size : 9.7GB
block_size : 262144
total_blocks : 37202
used_blocks : 37193
free_blocks : 9
wal_size : 0 bytes
memory_usage : 0 bytes
memory_limit : 13.4GB


In [4]:
q = cursor.execute('PRAGMA show_tables;')
res = q.fetchall()
res

[('raw_amzn_books_data',),
 ('raw_amzn_books_rating',),
 ('stg_amzn_books_data',),
 ('stg_amzn_books_rating',),
 ('vw_amzn_books_ratings',)]

In [5]:
q = cursor.execute("PRAGMA table_info('core.raw_amzn_books_rating');")
res = q.fetchall()
cols = list(zip(*res))[1]
cols

('Id',
 'Title',
 'Price',
 'User_id',
 'profileName',
 'review/helpfulness',
 'review/score',
 'review/time',
 'review/summary',
 'review/text')

In [6]:
q = cursor.execute("SELECT * FROM core.raw_amzn_books_rating LIMIT 100;")
res = q.fetchall()

In [7]:
df = pd.DataFrame(res, columns=list(cols))

In [8]:
df.head(3)

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."


In [9]:
q = cursor.execute("""
    SELECT 'raw_rating' AS tbl, COUNT(*) AS ct FROM core.raw_amzn_books_rating
    UNION ALL
    SELECT 'stg_rating' AS tbl, COUNT(*) AS ct FROM core.stg_amzn_books_rating
    UNION ALL
    SELECT 'view' AS tbl, COUNT(*) AS ct FROM core.vw_amzn_books_ratings;    
""")
res = q.fetchall()

In [10]:
res

[('raw_rating', 3000000), ('stg_rating', 2991226), ('view', 209)]

The view is currently the non-joined data!

```
, final AS (
    SELECT
        b.title AS data_title
        , r.title AS rating_title
        , b.description
        , b.authors
        , b.publisher
        , b.published_date
        , b.categories
        , b.ratings_count
        , r.price
        , r.user_id
    FROM books AS b
        FULL OUTER JOIN ratings AS r
            ON b.title = r.title
    WHERE 1 = 1
        AND b.title IS NULL
            OR r.title IS NULL
)
```

In [11]:
q = cursor.execute("PRAGMA table_info('core.vw_amzn_books_ratings');")
res = q.fetchall()
cols = list(zip(*res))[1]

In [12]:
q = cursor.execute("""SELECT * FROM core.vw_amzn_books_ratings;""")
res = q.fetchall()
df = pd.DataFrame(res, columns=list(cols))

# [13] saved a csv below to inspect - deleted cell

In [14]:
df

,data_title,rating_title,description,authors,publisher,published_date,categories,ratings_count,price,user_id
0,None,None,None,['Maharshi Ramana'],None,2015-12-15,None,1.0,NaN,None
1,None,None,None,None,None,None,None,NaN,NaN,A1M4HZYXTDAXY4
2,None,None,None,None,None,None,None,NaN,NaN,A2GQVG6F92CG72
3,None,None,None,None,None,None,None,NaN,NaN,A3B2Y8Q0Y5G2PE
4,None,None,None,None,None,None,None,NaN,NaN,A3A4CO8WPZ62E6
...,...,...,...,...,...,...,...,...,...,...
204,None,None,None,None,None,None,None,NaN,NaN,A37FFWZUGO8L7W
205,None,None,None,None,None,None,None,NaN,NaN,A1HDQHHA1AAK6O
206,None,None,None,None,None,None,None,NaN,NaN,A1VXG4DELT37C
207,None,None,None,None,None,None,None,NaN,NaN,A2TMUDWHQGS7K3


Basically the non-joined data is garbage, it didn't join because it's just `user_id` and mostly nothing else, some rows are even entirelly null.

This means we can safely ignore these 209 rows and just rely on the inner join as is.

Future checks will be to make sure the data isn't too scarce, look at nulls, etc.

In [15]:
conn.close()

---